In [1]:
import scipy.io
import pandas as pd

path = '/Users/raeed/Library/CloudStorage/OneDrive-UniversityofPittsburgh/0-projects/cst-rtt/cst-data/preTD/Dwight/2025-01-07/Dwight_2025-01-07_reward-type_medium_cst-rtt-dco_sorted.mat'

try:
    mat = scipy.io.loadmat(path, simplify_cells=True)
except NotImplementedError:
    try:
        import mat73
    except ImportError:
        raise ImportError("Must have mat73 installed to load mat73 files.")
    else:
        mat = mat73.loadmat(path)

real_keys = [k for k in mat.keys() if not (k.startswith("__") and k.endswith("__"))]
assert len(real_keys) == 1, "Multiple top-level keys found in mat file."
data_name = real_keys[0]
smile_data = mat[data_name]

# convert dict of list to list of dict (in case of mat73 file load)
if type(smile_data) is dict:
    smile_data = [dict(zip(smile_data.keys(), values)) for values in zip(*smile_data.values())]

/Users/raeed/codebase/0-projects/cst-dynamics/env/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [151]:
from src.extract_smile_data import compose_session_frame

tf = compose_session_frame(smile_data)
tf

channel                          monkey            date_time task   result  \
signal                                                                       
trial_id time                                                                
1        0 days 00:00:00         Dwight  2025-01-07 13:10:14  RTT  success   
         0 days 00:00:00.010000  Dwight  2025-01-07 13:10:14  RTT  success   
         0 days 00:00:00.020000  Dwight  2025-01-07 13:10:14  RTT  success   
         0 days 00:00:00.030000  Dwight  2025-01-07 13:10:14  RTT  success   
         0 days 00:00:00.040000  Dwight  2025-01-07 13:10:14  RTT  success   
...                                 ...                  ...  ...      ...   
152      0 days 00:00:04.340000  Dwight  2025-01-07 13:20:42  CST  success   
         0 days 00:00:04.350000  Dwight  2025-01-07 13:20:42  CST  success   
         0 days 00:00:04.360000  Dwight  2025-01-07 13:20:42  CST  success   
         0 days 00:00:04.370000  Dwight  2025-01-07 13:20:42  CST  success   
         0 days 00:00:04.380000  Dwight  2025-01-07 13:20:42  CST  success   

channel                                    state hand position              \
signal                                                       x           y   
trial_id time                                                                
1        0 days 00:00:00         Reach to Center     61.511639  796.000308   
         0 days 00:00:00.010000  Reach to Center     61.523815  796.008314   
         0 days 00:00:00.020000  Reach to Center     61.523992  796.014579   
         0 days 00:00:00.030000  Reach to Center     61.528734  796.005679   
         0 days 00:00:00.040000  Reach to Center     61.521756  795.999943   
...                                          ...           ...         ...   
152      0 days 00:00:04.340000          Success    131.019076  899.375467   
         0 days 00:00:04.350000          Success    130.678476  899.449719   
         0 days 00:00:04.360000          Success    130.328765  899.540967   
         0 days 00:00:04.370000          Success    129.983302  899.620873   
         0 days 00:00:04.380000          Success    129.675428  899.682388   

channel                                    motor cortex        ...          \
signal                                   z        ch6u1 ch9u1  ... ch122u2   
trial_id time                                                  ...           
1        0 days 00:00:00         45.378573            0     0  ...       0   
         0 days 00:00:00.010000  45.383986            0     1  ...       0   
         0 days 00:00:00.020000  45.373014            0     0  ...       1   
         0 days 00:00:00.030000  45.383058            0     0  ...       0   
         0 days 00:00:00.040000  45.400846            1     0  ...       0   
...                                    ...          ...   ...  ...     ...   
152      0 days 00:00:04.340000  11.113030            0     1  ...       0   
         0 days 00:00:04.350000  11.031885            0     0  ...       0   
         0 days 00:00:04.360000  10.946598            0     0  ...       0   
         0 days 00:00:04.370000  10.865358            0     1  ...       1   
         0 days 00:00:04.380000  10.795263            1     0  ...       0   

channel                                                                  \
signal                          ch122u3 ch122u4 ch123u1 ch123u2 ch124u1   
trial_id time                                                             
1        0 days 00:00:00              0       0       0       0       0   
         0 days 00:00:00.010000       0       0       0       0       0   
         0 days 00:00:00.020000       0       0       0       0       0   
         0 days 00:00:00.030000       0       0       0       0       0   
         0 days 00:00:00.040000       0       0       0       0       0   
...                                 ...     ...     ...     ...     ...   
152      0 days 00:00:04.340000       0       

In [205]:
import altair as alt

alt.data_transformers.enable(max_rows=10000)
raster = (
    alt.Chart(
        tf
        .loc[152,'motor cortex']
        .stack()
        .rename('spikes')
        .loc[lambda x: x > 0]
        .reset_index()
        .assign(time=lambda x: x['time'].dt.total_seconds())
    ).mark_tick().encode(
        x='time:Q',
        y=alt.Y('signal:N').axis(labels=False,ticks=False).title('Neuron'),
    )
    .configure_tick(
        bandSize=2,
        thickness=0.5,
        color='black',
    )
    .configure_axis(
        grid=False,
    )
    .properties(width=800, height=200)
)
tf.loc[1,'motor cortex'].stack().rename('spikes').loc[lambda x: x > 0]
raster

alt.Chart(...)

In [30]:
from src.extract_smile_data import get_smile_meta

temp = get_smile_meta(smile_data)
temp.groupby(['task','result']).size()
temp

,monkey,task,date_time,result
trial_id,,,,
1,Dwight,RTT,2025-01-07 13:10:14,success
2,Dwight,RTT,2025-01-07 13:10:18,success
3,Dwight,CST,2025-01-07 13:10:23,success
4,Dwight,DCO,2025-01-07 13:10:26,success
5,Dwight,DCO,2025-01-07 13:10:29,success
...,...,...,...,...
148,Dwight,RTT,2025-01-07 13:20:23,success
149,Dwight,CST,2025-01-07 13:20:28,success
150,Dwight,CST,2025-01-07 13:20:33,success


In [74]:
from src.extract_smile_data import get_trial_events,concat_trial_func_results

events=concat_trial_func_results(get_trial_events, smile_data)
(
    get_trial_events(smile_data[0])
    .reset_index(level='event')
    .set_index('time')
    .resample('10ms')
    .ffill()
    .rename(columns={'event':'state'})
    .squeeze()
)

time
0 days 00:00:00           Reach to Center
0 days 00:00:00.010000    Reach to Center
0 days 00:00:00.020000    Reach to Center
0 days 00:00:00.030000    Reach to Center
0 days 00:00:00.040000    Reach to Center
                               ...       
0 days 00:00:03.890000            Success
0 days 00:00:03.900000            Success
0 days 00:00:03.910000            Success
0 days 00:00:03.920000            Success
0 days 00:00:03.930000            Success
Freq: 10ms, Name: state, Length: 394, dtype: object

In [ ]:
from src.extract_smile_data import get_trial_spike_times,bin_spikes,concat_trial_func_results

spike_times = concat_trial_func_results(
    get_trial_spike_times,
    smile_data,
    keep_sorted_only=True,
)
bin_spikes(spike_times)
spike_times

In [23]:
smile_data[20]['TrialData']['stateTransitions'].T

array([[   1,    1],
       [   2,  478],
       [   3,  978],
       [   4, 1678],
       [   5, 1828],
       [   6, 2448],
       [   7, 2648],
       [   8, 2770],
       [  -1, 3147]], dtype=int16)

In [28]:
pd.DataFrame(smile_data[20]['Parameters']['StateTable']).columns
#pd.DataFrame(smile_data[0]['Parameters']['StateTable'])['stateName']
smile_data[21]['Parameters']['StateTable'][3]['stateName']

'Reach Target No Cheat'

In [82]:
import numpy as np
import altair as alt
from src.extract_smile_data import get_trial_hand_data,concat_trial_func_results

hand_data = concat_trial_func_results(
    get_trial_hand_data,
    smile_data,
    final_sampling_rate=100,
)

def sig_chart(df, **kwargs):
    return (
        alt.Chart(
            df
            .loc[:'0.2s']
            .reset_index()
            .assign(time=lambda x: x['time'].dt.total_seconds())
        )
        .mark_line(
            point='transparent',
            **kwargs
        )
        .encode(
            x='time:Q',
            y=alt.Y('x:Q').scale(zero=False),
        )
    )

(
    # sig_chart(marker_pos_df, color='black')
    # +
    sig_chart(hand_data.loc[1], color='blue')
)

alt.Chart(...)

In [86]:
1/pd.to_timedelta('10ms').total_seconds()

100.0

In [51]:
hand_data

x           y          z
trial_id time                                                     
1        0 days 00:00:00          61.508887  796.002094  45.378384
         0 days 00:00:00.001000   61.510622  795.999047  45.382433
         0 days 00:00:00.002000   61.512670  795.996987  45.385566
         0 days 00:00:00.003000   61.514943  795.996077  45.387669
         0 days 00:00:00.004000   61.517322  795.996311  45.388767
...                                     ...         ...        ...
152      0 days 00:00:04.379000  129.674946  899.680055  10.795821
         0 days 00:00:04.380000  129.641147  899.686611  10.787692
         0 days 00:00:04.381000  129.610379  899.692796  10.780259
         0 days 00:00:04.382000  129.583443  899.698484  10.773788
         0 days 00:00:04.383000  129.560914  899.703582  10.768479

[550176 rows x 3 columns]

In [ ]:
from src.extract_smile_data import get_spike_waveforms
from sklearn.decomposition import PCA
import altair as alt

waveforms = get_spike_waveforms(smile_data)

channel_waveforms = (
    waveforms
    .groupby('channel')
    .get_group(96)
    .rename(columns=lambda x: str(x))
)

channel_waveform_pca = pd.DataFrame(
    PCA(n_components=2)
    .fit_transform(
        channel_waveforms
    ),
    columns=['PC1','PC2'],
    index=channel_waveforms.index,
)

channel_waveforms = channel_waveforms.assign(**channel_waveform_pca)

sampled_waveforms = (
    channel_waveforms
    .sample(n=500)
    .reset_index()
)

# alt.data_transformers.enable('json')
selection = alt.selection_interval()

scatter = (
    alt.Chart(sampled_waveforms)
    .mark_point()
    .encode(
        x='PC1:Q',
        y='PC2:Q',
        color=alt.condition(
            selection,
            'unit:N',
            alt.value('lightgray'),
        ),
    )
    .add_selection(
        selection
    )
)

waveforms = (
    alt.Chart(sampled_waveforms)
    .mark_line()
    .encode(
        x='frame:Q',
        y='amplitude:Q',
        color=alt.condition(
            selection,
            'unit:N',
            alt.value('lightgray'),
        ),
        detail='snippet_id:O',
    )
    .transform_fold(
        [str(i) for i in range(30)],
        as_=['snippet_frame','amplitude'],
    )
    .transform_calculate(
        frame='toNumber(datum.snippet_frame)',
    )
)
scatter | waveforms

In [150]:
tf

channel                          monkey            date_time task   result  \
signal                                                                       
trial_id time                                                                
1        0 days 00:00:00         Dwight  2025-01-07 13:10:14  RTT  success   
         0 days 00:00:00.010000  Dwight  2025-01-07 13:10:14  RTT  success   
         0 days 00:00:00.020000  Dwight  2025-01-07 13:10:14  RTT  success   
         0 days 00:00:00.030000  Dwight  2025-01-07 13:10:14  RTT  success   
         0 days 00:00:00.040000  Dwight  2025-01-07 13:10:14  RTT  success   
...                                 ...                  ...  ...      ...   
152      0 days 00:00:04.340000  Dwight  2025-01-07 13:20:42  CST  success   
         0 days 00:00:04.350000  Dwight  2025-01-07 13:20:42  CST  success   
         0 days 00:00:04.360000  Dwight  2025-01-07 13:20:42  CST  success   
         0 days 00:00:04.370000  Dwight  2025-01-07 13:20:42  CST  success   
         0 days 00:00:04.380000  Dwight  2025-01-07 13:20:42  CST  success   

channel                                    state hand position              \
signal                                                       x           y   
trial_id time                                                                
1        0 days 00:00:00         Reach to Center     61.511639  796.000308   
         0 days 00:00:00.010000  Reach to Center     61.523815  796.008314   
         0 days 00:00:00.020000  Reach to Center     61.523992  796.014579   
         0 days 00:00:00.030000  Reach to Center     61.528734  796.005679   
         0 days 00:00:00.040000  Reach to Center     61.521756  795.999943   
...                                          ...           ...         ...   
152      0 days 00:00:04.340000          Success    131.019076  899.375467   
         0 days 00:00:04.350000          Success    130.678476  899.449719   
         0 days 00:00:04.360000          Success    130.328765  899.540967   
         0 days 00:00:04.370000          Success    129.983302  899.620873   
         0 days 00:00:04.380000          Success    129.675428  899.682388   

channel                                    motor cortex        ...          \
signal                                   z        ch6u1 ch9u1  ... ch122u2   
trial_id time                                                  ...           
1        0 days 00:00:00         45.378573            0     0  ...       0   
         0 days 00:00:00.010000  45.383986            0     1  ...       0   
         0 days 00:00:00.020000  45.373014            0     0  ...       1   
         0 days 00:00:00.030000  45.383058            0     0  ...       0   
         0 days 00:00:00.040000  45.400846            1     0  ...       0   
...                                    ...          ...   ...  ...     ...   
152      0 days 00:00:04.340000  11.113030            0     1  ...       0   
         0 days 00:00:04.350000  11.031885            0     0  ...       0   
         0 days 00:00:04.360000  10.946598            0     0  ...       0   
         0 days 00:00:04.370000  10.865358            0     1  ...       1   
         0 days 00:00:04.380000  10.795263            1     0  ...       0   

channel                                                                  \
signal                          ch122u3 ch122u4 ch123u1 ch123u2 ch124u1   
trial_id time                                                             
1        0 days 00:00:00              0       0       0       0       0   
         0 days 00:00:00.010000       0       0       0       0       0   
         0 days 00:00:00.020000       0       0       0       0       0   
         0 days 00:00:00.030000       0       0       0       0       0   
         0 days 00:00:00.040000       0       0       0       0       0   
...                                 ...     ...     ...     ...     ...   
152      0 days 00:00:04.340000       0       